In [1]:
import numpy as np
from importlib import reload
from matplotlib import pyplot as plt
import pandas as pd

try:
    reload  # Python 2.7
except NameError:
    try:
        from importlib import reload  # Python 3.4+
    except ImportError:
        from imp import reload  # Python 3.0 - 3.3

import sys
sys.path.append("/booleanfs2/sahoo/Hegemon/")
sys.path = ["/booleanfs2/sahoo/BoNE/"] + sys.path

import StepMiner as smn
import HegemonUtil as hu

import MacUtils as mut
reload(mut)

%matplotlib inline

from convert_gene_mm import getGroupsHs, getGroupsMm

In [2]:
# Codes for Boolean Relationships:
# 0 - No relation
# 1 - X low -> i high
# 2 - X low -> i low
# 3 - X high -> i high
# 4 - X high -> i low
# 5 - Equivalent
# 6 - Opposite

def get_type(rel):
    if rel == 1:
        return 'Lo-Hi'
    elif rel == 2:
        return 'Lo-Lo'
    elif rel == 3:
        return 'Hi-Hi'
    elif rel == 4:
        return 'Hi-Lo'
    elif rel == 5:
        return 'Equiv'
    elif rel == 6:
        return 'Oppos'
    else:
        return ''


def get_relationship(geneA, geneB, sthr, pthr):
    ids1 = list(h.getIDs(geneA).keys())
    ids2 = list(h.getIDs(geneB).keys())
    if not ids1 or not ids2:
        return -1, None, None
    #id1 = ids1[0]
    #id2 = ids2[0]
    best_corr_coef = 1
    for id1 in ids1:
        for id2 in ids2:
            #print(id1)
            #print(id2)
            e1 = h.getExprData(id1)
            e2 = h.getExprData(id2)
            thrx = hu.getThrData(e1, 2, len(e1) - 1)
            thry = hu.getThrData(e2, 2, len(e2) - 1)
            low = h.getArraysAll(id1, thrx[2], "lo")
            high = h.getArraysAll(id1, thrx[3], "hi")
            a_high, a_med = h.getBitVector(low, high, arr=None)
            low = h.getArraysAll(id2, thry[2], "lo")
            high = h.getArraysAll(id2, thry[3], "hi")
            b_high, b_med = h.getBitVector(low, high, arr=None)

            bs = hu.getBooleanStats(a_high, a_med, b_high, b_med)
            rel, stats = hu.getBooleanRelationType(bs, sthr, pthr)
            rel_type = get_type(rel)

            expr1 = [float(x) for x in e1[2:len(e1)]]
            expr2 = [float(x) for x in e2[2:len(e2)]]
            corr_coef = np.corrcoef(expr1, expr2)[0,1]  

            if corr_coef <= best_corr_coef:
                best_corr_coef = corr_coef
                best_id1 = id1
                best_id2 = id2
                best_rel_type = rel_type
                best_stats = stats
                best_corr_coef = corr_coef
            
    return best_rel_type, best_stats, best_corr_coef

In [4]:
reload(mut)
wt1, l1 = mut.getCls13a14a3(dir_name='./data/SMaRT_Genes/')
#wt1, l1 = mut.getCls13()
#wt1, l1 = mut.getCls14a3()
#l1 = mut.getGroupsMm(l1)
#l1 = mut.getGroupsMmv1(l1)
ana = mut.MacAnalysis()
ana.getGEOMacAnn()
#ana.getBraun2018()
ana.orderDataDf(l1, wt1)
#ana.normGene("DBP")
#fig = mut.plotViolinBar(ana, "Mac")
#ana.getMacMetrics(ana.cval[0])

Human Macrophages 2012 (n = 197)
GEO http://hegemon.ucsd.edu/Tools/explore.php?key=gbm&id=G16
68 [47, 13, 8] GEO http://hegemon.ucsd.edu/Tools/explore.php?key=gbm&id=G16 G16
[84, 305, 313]


In [6]:
db = hu.Database("/booleanfs2/sahoo/Hegemon/explore.conf")
dbid = "G16"
h = hu.Hegemon(db.getDataset(dbid))
h.init()
h.initPlatform()
h.initSurv()

In [7]:
#read Keller2009 PNAS geness (~1400 circadian genes, obtained using Fourier statistics)
file_path = './data/keller2009pnas_CR_genes.txt'

with open(file_path, 'r') as file:
    keller_genes_cr = [line.strip() for line in file.readlines()]
print(keller_genes_cr)

['1500010J02Rik', 'Abcc3', 'Acy1', 'Ahsa1', 'AI586015', 'Ang', 'Arntl', 'Arpc4', 'Arsg', 'Atp2b1', 'Blvra', 'Camsap1l1', 'Car6', 'Ccl24', 'Cd86', 'Cdca7', 'Clec5a', 'Coro7', 'Creld2', 'Cry1', 'Ctsl', 'Ddx1', 'Dynlt1', 'Fbxl20', 'Fkbp4', 'Gbas', 'Gstm4', 'Hnrpl', 'Hsp110', 'Hspa5', 'Ints7', 'Ints9', 'Irgq', 'Jun', 'LOC100043257', 'Lpl', 'Mbnl1', 'Msi2', 'Mxd4', 'Nfil3', 'Nln', 'Nr1d2', 'P2ry14', 'P4ha1', 'Pdlim1', 'Per2', 'Pgk1', 'Plau', 'Rfx1', 'Sgsh', 'Slc25a1', 'Slc7a8', 'Sort1', 'Stip1', 'Tfdp1', 'Ubl5', 'Vamp8', 'Yipf2', 'Zmym3', 'Znhit4', 'Abcb8', 'Abi1', 'Adam17', 'Agpat3', 'Ahsa2', 'AI427809', 'Alkbh3', 'Antxr2', 'Aqp9', 'Araf', 'Atxn7l1', 'B4galt6', 'BC032204', 'Bcl2a1c', 'Bmyc', 'Cd180', 'Cd59a', 'Cnot6', 'Cpt1a', 'Cyb5r3', 'Cyp27a1', 'Cysltr1', 'Dcp1b', 'Dctn6', 'Depdc6', 'Dpysl3', 'Glipr2', 'Glt25d1', 'Gpsn2', 'Gsr', 'Khk', 'LOC100043719', 'LOC668181', 'Mast3', 'Mgst1', 'Ncoa7', 'Nme1', 'Nr2c2', 'Parc', 'Pde1b', 'Per3', 'Pigt', 'Prkacb', 'Rara', 'Rbm3', 'Rfx2', 'Senp6', 'Slc

In [55]:
#overlay the Keller 1400 genes on the SMaRT map
SMaRT_genes = {}
SMaRT_genes_Mm = {}
SMaRT_Keller_genes = {}
for i in range(1,15):
    with open(f'./data/SMaRT_Genes/node-{i}.txt') as f:
        SMaRT_genes[i] = f.read().splitlines()
        SMaRT_genes_Mm[i] = getGroupsMm([set(SMaRT_genes[i])])
        SMaRT_Keller_genes[i] = list(set(keller_genes_cr).intersection(set(SMaRT_genes_Mm[i][0])))
        print(f'C#{i}: {SMaRT_Keller_genes[i]}\n')
        output_file = f'./Results/SMaRT_Mm_Keller2009_1400cr_genes/node{i}.txt'
        with open(output_file, 'w') as out_f:
            for gene in SMaRT_Keller_genes[i]:
                out_f.write(gene + '\n')

print('-'*100)
for i in range(1,15):
    print(f'Cluster {i:2d}: {len(SMaRT_Keller_genes[i])} (/{len(SMaRT_genes_Mm[i][0])})')

C#1: ['Rere', 'Itpr3', 'Gpc1', 'Mst1r', 'Ybx1', 'Thra', 'Rasa3', 'Grk5', 'Arfrp1', 'Esr1', 'Selp', 'Dock6', 'Hip1', 'Ncaph2', 'Pkmyt1', 'Kctd2', 'Tnfrsf8', 'Zer1', 'Dzip1', 'Prm2', 'Klf9', 'Adarb1', 'Entpd4', 'Pik3cd', 'Hs6st1', 'Ccr10', 'Wdtc1', 'Soat2', 'Sphk2', 'Srpk2', 'Foxn3', 'Slc12a4', 'Pdlim4', 'Icosl', 'Gnb1l', 'Cdk10', 'Fgfr1', 'Scamp2', 'Casp2', 'Tom1l2', 'Ube2i', 'Rara', 'Vps45', 'Rap1gap', 'Rnf126', 'Myo9a', 'Rrp12', 'Sardh', 'Tbc1d2b', 'Pde1b', 'Efnb2', 'Nr1d1', 'Stk38', 'Tbc1d9b', 'Slc12a5', 'Etv1', 'Hgf', 'Nadk', 'Mbd2', 'Lgals9', 'Zc3h13', 'Fbxl2', 'Mmp19', 'Tbcd', 'Pax5', 'Dnaja4', 'Arg1', 'Agrp', 'Rarb', 'Cnr2', 'Gfer', 'Med16', 'Arhgef12', 'Pqbp1', 'Arid1a', 'Atp2a1', 'Tbxa2r', 'Sqstm1', 'Fastk', 'Polg', 'Gmds', 'Tubb3', 'Rbm19', 'Rfx2', 'Cd79a', 'Ivd', 'Inpp4a', 'Map2k7', 'Cdkn1c', 'Ctbp2', 'Rprm', 'Ppp2r1a', 'Crip2', 'Lypla2', 'Cln6', 'Smo', 'Aldh6a1', 'Lmna', 'Ehmt2', 'Med24', 'Tnks', 'Padi4', 'Tppp3', 'Pdk2', 'Scara3', 'Rangap1', 'Khk', 'Nfix', 'Slc22a17', 'Pnpl

In [54]:
#Keller genes in the SMaRT-HS map
SMaRT_Keller_genes_Hs = {}
for i in range(1,15):
    SMaRT_Keller_genes_Hs[i] = getGroupsHs([set(SMaRT_Keller_genes[i])])[0]
    print(f'C#{i}: {SMaRT_Keller_genes_Hs[i]}\n')
    output_file = f'./Results/SMaRT_Hs_Keller2009_1400cr_genes/node{i}.txt'
    with open(output_file, 'w') as out_f:
        for gene in SMaRT_Keller_genes_Hs[i]:
            out_f.write(gene + '\n')

print('-'*100)
for i in range(1,15):
    print(f'Cluster {i:2d}: {len(SMaRT_Keller_genes_Hs[i])} (/{len(SMaRT_genes[i][0])})')

C#1: {'RPRM', 'CREBBP', 'SRPK3', 'SMG6', 'SIGLEC12', 'HIP1', 'TNFRSF8', 'NCAPH2', 'TBCD', 'HGF', 'SLC22A17', 'RBM19', 'KLHDC8A', 'FOXN3', 'CA5B', 'GFER', 'RERE', 'TUBGCP2', 'SLC9A3R2', 'ARG1', 'CDK10', 'SMO', 'GPC1', 'MST1R', 'TBC1D9B', 'UBE2I', 'NUDC', 'PIK3CD', 'XDH', 'WDTC1', 'TOM1L2', 'CENPA', 'FGFR1', 'PKMYT1', 'RARB', 'TOMM40', 'FP565260.3', 'CLN6', 'ATP2A1', 'PDE1B', 'YIPF2', 'KHK', 'RANGAP1', 'LGALS16', 'SARDH', 'LGALS13', 'MED16', 'GPRC5B', 'OBSL1', 'DBP', 'ENTPD4', 'CTBP2', 'FASTK', 'LSM14B', 'DOCK6', 'RFX2', 'CCR10', 'SLC12A4', 'FBXL2', 'CD79A', 'CDK2', 'VPS45', 'STK16', 'NR1D1', 'RARA', 'MBD2', 'MMP19', 'CASP2', 'MAP2K7', 'SIGLEC8', 'SLC12A5', 'SELP', 'TBXA2R', 'SPHK2', 'PPP2R1A', 'ADARB1', 'STK38', 'H6PD', 'ARFRP1', 'RAP1GAP', 'ZC3H13', 'YBX1', 'SQSTM1', 'KLF9', 'DNAJA4', 'PDK2', 'GMDS', 'HS6ST1', 'AGRP', 'TEF', 'ESR1', 'FXYD7', 'RRP12', 'SIGLEC7', 'ETV1', 'AATK', 'SRPK2', 'AC092143.1', 'CRIP2', 'THRA', 'MOSPD3', 'PRM2', 'CD22', 'RNF126', 'SIGLEC9', 'NADK', 'TBC1D2B', 'CDK

In [27]:
#Compute Boolean Relationships
S_thr = 4.5
p_thr = 0.1

data_list = []
c1_genes_hs = getGroupsHs([set(SMaRT_Keller_genes[1])])[0]
for gene in SMaRT_genes[14]:
    cnt = 0
    for c1_gene in c1_genes_hs:
        rel,_,corrcoef = get_relationship(c1_gene, gene, S_thr, p_thr)
        if rel in {'Hi-Lo'}:
            cnt += 1
    print(f'{gene} in SMaRT_C#14 has {cnt} Hi-Lo relations from SMaRT_Keller_C#1.')
    data = {'C#14 Gene': gene, 'Relation':'Hi-Lo', 'Count':cnt}
    data_list.append(data)

C12orf29 in SMaRT_C#14 has 17 Hi-Lo relations from SMaRT_Keller_C#1.
RPL22 in SMaRT_C#14 has 0 Hi-Lo relations from SMaRT_Keller_C#1.
KTELC1 in SMaRT_C#14 has 0 Hi-Lo relations from SMaRT_Keller_C#1.
MAP3K8 in SMaRT_C#14 has 8 Hi-Lo relations from SMaRT_Keller_C#1.
DCUN1D4 in SMaRT_C#14 has 4 Hi-Lo relations from SMaRT_Keller_C#1.
DIS3 in SMaRT_C#14 has 0 Hi-Lo relations from SMaRT_Keller_C#1.
SLC38A7 in SMaRT_C#14 has 0 Hi-Lo relations from SMaRT_Keller_C#1.
EIF3EIP in SMaRT_C#14 has 0 Hi-Lo relations from SMaRT_Keller_C#1.
CPEB3 in SMaRT_C#14 has 0 Hi-Lo relations from SMaRT_Keller_C#1.
RBBP8 in SMaRT_C#14 has 1 Hi-Lo relations from SMaRT_Keller_C#1.
DFFB in SMaRT_C#14 has 0 Hi-Lo relations from SMaRT_Keller_C#1.
EBI2 in SMaRT_C#14 has 2 Hi-Lo relations from SMaRT_Keller_C#1.
ZNF274 in SMaRT_C#14 has 0 Hi-Lo relations from SMaRT_Keller_C#1.
C19orf42 in SMaRT_C#14 has 0 Hi-Lo relations from SMaRT_Keller_C#1.
RPL13 in SMaRT_C#14 has 0 Hi-Lo relations from SMaRT_Keller_C#1.
ZBTB5 in SMa

KCTD3 in SMaRT_C#14 has 3 Hi-Lo relations from SMaRT_Keller_C#1.
STK38 in SMaRT_C#14 has 0 Hi-Lo relations from SMaRT_Keller_C#1.
CRK in SMaRT_C#14 has 0 Hi-Lo relations from SMaRT_Keller_C#1.
RPL15 in SMaRT_C#14 has 0 Hi-Lo relations from SMaRT_Keller_C#1.
NARS2 in SMaRT_C#14 has 0 Hi-Lo relations from SMaRT_Keller_C#1.
CRKRS in SMaRT_C#14 has 0 Hi-Lo relations from SMaRT_Keller_C#1.
IPO5 in SMaRT_C#14 has 0 Hi-Lo relations from SMaRT_Keller_C#1.
TFPI in SMaRT_C#14 has 17 Hi-Lo relations from SMaRT_Keller_C#1.
CCDC88A in SMaRT_C#14 has 24 Hi-Lo relations from SMaRT_Keller_C#1.
CAMK1D in SMaRT_C#14 has 0 Hi-Lo relations from SMaRT_Keller_C#1.
TTC31 in SMaRT_C#14 has 0 Hi-Lo relations from SMaRT_Keller_C#1.
RPL3 in SMaRT_C#14 has 0 Hi-Lo relations from SMaRT_Keller_C#1.


In [36]:
rel_df = pd.DataFrame(data_list)
rel_df = rel_df.sort_values(by='Count', ascending=False)
rel_df.to_csv('SMaRT_Keller_C1-HiLo-C14_sthr=4.5_pthr=0.1.csv', index=False)

In [44]:
rel_df = pd.read_csv('SMaRT_Keller_C1-HiLo-C14_sthr=4.5_pthr=0.1.csv')
rel_df

,C#14 Gene,Relation,Count
0,CCDC88A,Hi-Lo,24
1,C12orf29,Hi-Lo,17
2,TFPI,Hi-Lo,17
3,LOC730092,Hi-Lo,17
4,FCF1,Hi-Lo,14
...,...,...,...
132,ZNF187,Hi-Lo,0
133,ZNF23,Hi-Lo,0
134,RPL10,Hi-Lo,0
135,ZNF266,Hi-Lo,0


In [38]:
#Compute Boolean Relationships
S_thr = 3.0
p_thr = 0.1

data_list = []
c1_genes_hs = getGroupsHs([set(SMaRT_Keller_genes[1])])[0]
for gene in SMaRT_genes[14]:
    cnt = 0
    for c1_gene in c1_genes_hs:
        rel,_,corrcoef = get_relationship(c1_gene, gene, S_thr, p_thr)
        if rel in {'Hi-Lo'}:
            cnt += 1
    print(f'{gene} in SMaRT_C#14 has {cnt} Hi-Lo relations from SMaRT_Keller_C#1.')
    data = {'C#14 Gene': gene, 'Relation':'Hi-Lo', 'Count':cnt}
    data_list.append(data)

C12orf29 in SMaRT_C#14 has 94 Hi-Lo relations from SMaRT_Keller_C#1.
RPL22 in SMaRT_C#14 has 13 Hi-Lo relations from SMaRT_Keller_C#1.
KTELC1 in SMaRT_C#14 has 39 Hi-Lo relations from SMaRT_Keller_C#1.
MAP3K8 in SMaRT_C#14 has 107 Hi-Lo relations from SMaRT_Keller_C#1.
DCUN1D4 in SMaRT_C#14 has 98 Hi-Lo relations from SMaRT_Keller_C#1.
DIS3 in SMaRT_C#14 has 67 Hi-Lo relations from SMaRT_Keller_C#1.
SLC38A7 in SMaRT_C#14 has 3 Hi-Lo relations from SMaRT_Keller_C#1.
EIF3EIP in SMaRT_C#14 has 48 Hi-Lo relations from SMaRT_Keller_C#1.
CPEB3 in SMaRT_C#14 has 90 Hi-Lo relations from SMaRT_Keller_C#1.
RBBP8 in SMaRT_C#14 has 99 Hi-Lo relations from SMaRT_Keller_C#1.
DFFB in SMaRT_C#14 has 2 Hi-Lo relations from SMaRT_Keller_C#1.
EBI2 in SMaRT_C#14 has 72 Hi-Lo relations from SMaRT_Keller_C#1.
ZNF274 in SMaRT_C#14 has 38 Hi-Lo relations from SMaRT_Keller_C#1.
C19orf42 in SMaRT_C#14 has 2 Hi-Lo relations from SMaRT_Keller_C#1.
RPL13 in SMaRT_C#14 has 4 Hi-Lo relations from SMaRT_Keller_C#1.
Z

MLL in SMaRT_C#14 has 8 Hi-Lo relations from SMaRT_Keller_C#1.
KCTD3 in SMaRT_C#14 has 92 Hi-Lo relations from SMaRT_Keller_C#1.
STK38 in SMaRT_C#14 has 37 Hi-Lo relations from SMaRT_Keller_C#1.
CRK in SMaRT_C#14 has 29 Hi-Lo relations from SMaRT_Keller_C#1.
RPL15 in SMaRT_C#14 has 2 Hi-Lo relations from SMaRT_Keller_C#1.
NARS2 in SMaRT_C#14 has 2 Hi-Lo relations from SMaRT_Keller_C#1.
CRKRS in SMaRT_C#14 has 2 Hi-Lo relations from SMaRT_Keller_C#1.
IPO5 in SMaRT_C#14 has 1 Hi-Lo relations from SMaRT_Keller_C#1.
TFPI in SMaRT_C#14 has 109 Hi-Lo relations from SMaRT_Keller_C#1.
CCDC88A in SMaRT_C#14 has 103 Hi-Lo relations from SMaRT_Keller_C#1.
CAMK1D in SMaRT_C#14 has 2 Hi-Lo relations from SMaRT_Keller_C#1.
TTC31 in SMaRT_C#14 has 10 Hi-Lo relations from SMaRT_Keller_C#1.
RPL3 in SMaRT_C#14 has 2 Hi-Lo relations from SMaRT_Keller_C#1.


,C#14 Gene,Relation,Count
133,CCDC88A,Hi-Lo,24
0,C12orf29,Hi-Lo,17
132,TFPI,Hi-Lo,17
50,LOC730092,Hi-Lo,17
51,FCF1,Hi-Lo,14
...,...,...,...
122,SLC9A8,Hi-Lo,0
123,USE1,Hi-Lo,0
124,MLL,Hi-Lo,0
113,TCEA1,Hi-Lo,0


In [40]:
rel_df = pd.DataFrame(data_list)
rel_df = rel_df.sort_values(by='Count', ascending=False)
rel_df.to_csv('SMaRT_Keller_C1-HiLo-C14_sthr=3.0_pthr=0.1.csv', index=False)

In [43]:
rel_df.head(10)

,C#14 Gene,Relation,Count
132,TFPI,Hi-Lo,109
3,MAP3K8,Hi-Lo,107
18,FRY,Hi-Lo,106
30,PIBF1,Hi-Lo,105
120,SFRS15,Hi-Lo,103
133,CCDC88A,Hi-Lo,103
97,PARP2,Hi-Lo,99
9,RBBP8,Hi-Lo,99
36,CLEC4A,Hi-Lo,98
4,DCUN1D4,Hi-Lo,98
